## Connect to server and start jupyter-docker-desktop

Aim: these are my experiments for starting a jupyter-docker-desktop on a remote server via ssh. 
The idea is to turn this into a little gui.

The GUI will need input fields for the HOSTIP, USER, and PRIVATEKEYFILE
as well as buttons for

* Connect
* Start Server
* Kill Server
* Close Connection

There will also need to be an output field for the URL.

On the server, the user needs to have the following `rundocker.sh` file in their home directory:

```
#!/bin/bash
FREEPORT=`/usr/bin/python3 -c 'import socket; s=socket.socket(); s.bind(("", 0)); print(s.getsockname()[1]); s.close()'`
echo $FREEPORT > .jupyter_docker_port
docker run --rm  -p 10.11.19.188:$FREEPORT:8888 -v /home/spacem:/spacem_home spacem/jupyter-desktop-server 2> .jupyter_docker_stderr

```

They also need to have a public key corresponding to their private key on the server.

In [1]:
import paramiko
import time
import re

In [2]:
HOSTIP = "10.11.19.188"
USER = "spacem"
PRIVATEKEYFILE = "/home/hilsenst/.ssh/id_rsa"

DOCKER_START_CMD = '$HOME/rundocker.sh'
DOCKER_STDERR_CMD =  "/usr/bin/cat $HOME/.jupyter_docker_stderr"
DOCKER_PORT_CMD = "/usr/bin/cat $HOME/.jupyter_docker_port"
DOCKER_KILL_CMD = "/usr/bin/killall docker"

SLEEPTIME = 5

This is the init part that sets up the connection info:

In [3]:
k = paramiko.RSAKey.from_private_key_file(PRIVATEKEYFILE)
c = paramiko.SSHClient()
c.set_missing_host_key_policy(paramiko.AutoAddPolicy())

This will need to be bound to the _Connect_ button

In [4]:
c.connect( hostname = HOSTIP, username = USER, pkey = k )

This will need to be bound to the _Start Server_ button

In [5]:
stdin , stdout, stderr = c.exec_command(DOCKER_START_CMD)
time.sleep(SLEEPTIME) # give server some time to start up

# Jupyterlab sends connection URL to stderr, which
# we redirect into a file in the startup script on 
# the server. 
stdin , stdout, stderr = c.exec_command(DOCKER_STDERR_CMD)
catout = stdout.read()
for line in catout.decode('utf-8').split("\n"):
    m = re.match(r'.*(?P<token>\?token=.+)', line)
    if m is not None:
        break
token = m["token"]

# The startup script on the server looks for an
# available port number and redirects it into a file
stdin , stdout, stderr = c.exec_command(DOCKER_PORT_CMD)
port = stdout.read().decode('utf-8').strip()

print(f"http://{HOSTIP}:{port}/{token}")

http://10.11.19.188:35727/?token=1216b0a1d24793882cb0abd09eaa0e9f8ecfd7ad672b888e


This will need to be bound to the _Kill Server_ button

In [ ]:
stdin , stdout, stderr = c.exec_command(DOCKER_KILL_CMD)

This will need to be bound to the _Close Connection_ button

In [ ]:
c.close()